In [1]:
# from translate import Translator
from langdetect import detect
from dateutil import parser
import re
import psycopg2
import pandas as pd

import translators as ts
from os import environment

/var/folders/tg/c_k1syfs2b9gvpkyjkcmdqx00000gn/T/ipykernel_20354/3878500887.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
use_date_str = True

In [3]:
conn = psycopg2.connect(
host = environment['DB_HOST'],
port = environment['DB_PORT'],
user = environment['DB_USER'],
password = environment['DB_PASSWORD'],
database = environemnt['DB_NAME'])

In [4]:
offset = 7
limit = 500

if use_date_str == False:
    query = r"""
        SELECT id, raw_html, language FROM articles
        WHERE article_type = 'blog'
        AND parsed_date IS NULL
        LIMIT {}
    ;""".format(limit)
else:
    query = r"""
        SELECT id, raw_html, posted_date_str, language FROM articles
        WHERE posted_date_str IS NOT NULL
        AND parsed_date IS NULL
        LIMIT {}
    ;""".format(limit)
body = pd.read_sql_query(query, conn)
body

/var/folders/tg/c_k1syfs2b9gvpkyjkcmdqx00000gn/T/ipykernel_20354/4224385000.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  body = pd.read_sql_query(query, conn)


,id,raw_html,posted_date_str,language
0,9606,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2022-Ж., 22-FEBRUARY",ky
1,9616,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2021-Ж., 2-DECEMBER",ky
2,9627,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2023-Ж., 27-OCTOBER",ky
3,9628,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2023-Ж., 6-OCTOBER",ky
4,9618,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2023-Ж., 19-JULY",ky
...,...,...,...,...
125,4818,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...","JUNE 15, 2020",en
126,1285,"<html lang=""ky"" dir=""ltr"" class="" js""><head>\n...","2020-Ж., 14-JULY",ky
127,1275,"<html lang=""ky"" dir=""ltr"" class="" js""><head>\n...","2022-Ж., 15-FEBRUARY",ky
128,9605,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2021-Ж., 29-APRIL",ky


In [5]:
# reg = r"(?<=>)(.*?)(?=<\/)"
reg = r"<p>(.*?)<\/p>"
reg_date = r'<h6 class="posted-date">(.*?)</h6>'

body['raw'] = body['raw_html'].apply(lambda x: ' '.join(re.findall(reg, x)))

if use_date_str == False:
    body['date'] = body['raw_html'].apply(lambda x: ' '.join(re.findall(reg_date, x)).strip().lower().replace(r'[\s\n]+', ' '))
else:
    body['date'] = body['posted_date_str']

In [6]:
def verifyLanguage (row):
    lang = row['language']
    if lang == 'en':
        try:
            return detect(row['raw'])
        except:
            return lang
    else:
        return lang

In [7]:
def translate (row):
    lang = row['verified_language']
    # translator = Translator(from_lang = lang, to_lang = 'en')
    # return translator.translate(row['date'].strip().upper())
    if lang != 'en':
        try:
            return ts.translate_text(row['date'], from_language = lang, to_language = 'en')
        except:
            return row['date']
    else:
        return row['date']

In [8]:
def parseDate (row):
    try:
        return parser.parse(row['date_translation'])
    except:
        return None

In [9]:
body['verified_language'] = body.apply(verifyLanguage, axis = 1)
body['date_translation'] = body.apply(translate, axis = 1)
body['datetime'] = body.apply(parseDate, axis = 1)
body

,id,raw_html,posted_date_str,language,raw,date,verified_language,date_translation,datetime
0,9606,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2022-Ж., 22-FEBRUARY",ky,United Nati...,"2022-Ж., 22-FEBRUARY",ky,"2022-J., 22-FEBRUARY",None
1,9616,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2021-Ж., 2-DECEMBER",ky,Чынаранын столунда предметтердин уникалдуу к...,"2021-Ж., 2-DECEMBER",ky,"2021-J., 2-DECEMBER",None
2,9627,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2023-Ж., 27-OCTOBER",ky,"&nbsp; <strong>Бишкек, 27 – октябрь, 2023-жыл<...","2023-Ж., 27-OCTOBER",ky,"2023-J., October 27",None
3,9628,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2023-Ж., 6-OCTOBER",ky,"&nbsp; <strong>""STEM4ALL кесиптердин келечеги ...","2023-Ж., 6-OCTOBER",ky,"2023-J., 6October",None
4,9618,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2023-Ж., 19-JULY",ky,<strong>Бишкек – 19-июль 2023-жыл –</strong> Б...,"2023-Ж., 19-JULY",ky,"2023-J., July 19",None
...,...,...,...,...,...,...,...,...,...
125,4818,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...","JUNE 15, 2020",en,"Rapid urbanisation, increasing air pollution, ...","JUNE 15, 2020",en,"JUNE 15, 2020",2020-06-15 00:00:00
126,1285,"<html lang=""ky"" dir=""ltr"" class="" js""><head>\n...","2020-Ж., 14-JULY",ky,"Рена Султангазиева, Inno KG сынагынын жеңүүчү...","2020-Ж., 14-JULY",ky,"2020-J., July 14",None
127,1275,"<html lang=""ky"" dir=""ltr"" class="" js""><head>\n...","2022-Ж., 15-FEBRUARY",ky,<i>ololoOsh креативдүү борборунун базасында ...,"2022-Ж., 15-FEBRUARY",ky,"2022-J., 15-FEBRUARY",None
128,9605,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2021-Ж., 29-APRIL",ky,Сүрөт: жеке архив Бегимай Талас шаарында туу...,"2021-Ж., 29-APRIL",ky,"2021-J., 29-APRIL",None


In [10]:
body[body['datetime'].isna()]

,id,raw_html,posted_date_str,language,raw,date,verified_language,date_translation,datetime
0,9606,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2022-Ж., 22-FEBRUARY",ky,United Nati...,"2022-Ж., 22-FEBRUARY",ky,"2022-J., 22-FEBRUARY",None
1,9616,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2021-Ж., 2-DECEMBER",ky,Чынаранын столунда предметтердин уникалдуу к...,"2021-Ж., 2-DECEMBER",ky,"2021-J., 2-DECEMBER",None
2,9627,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2023-Ж., 27-OCTOBER",ky,"&nbsp; <strong>Бишкек, 27 – октябрь, 2023-жыл<...","2023-Ж., 27-OCTOBER",ky,"2023-J., October 27",None
3,9628,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2023-Ж., 6-OCTOBER",ky,"&nbsp; <strong>""STEM4ALL кесиптердин келечеги ...","2023-Ж., 6-OCTOBER",ky,"2023-J., 6October",None
4,9618,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2023-Ж., 19-JULY",ky,<strong>Бишкек – 19-июль 2023-жыл –</strong> Б...,"2023-Ж., 19-JULY",ky,"2023-J., July 19",None
5,1290,"<html lang=""ky"" dir=""ltr"" class="" js""><head>\n...","2022-Ж., 3-FEBRUARY",ky,<i>БУУӨПтүн Кыргызстандагы өкүлчүлүгү коммун...,"2022-Ж., 3-FEBRUARY",ky,"2022-J., 3-FEBRUARY",None
6,1287,"<html lang=""ky"" dir=""ltr"" class="" js""><head>\n...","2021-Ж., 25-FEBRUARY",ky,"""ДИЛГИР"" тиркемесинин логотиби . Сүрөт: ""Техн...","2021-Ж., 25-FEBRUARY",ky,"2021-J., 25-FEBRUARY",None
7,1288,"<html lang=""ky"" dir=""ltr"" class="" js""><head>\n...","2019-Ж., 13-DECEMBER",ky,Заманбап технологиялар заманында интернет-ме...,"2019-Ж., 13-DECEMBER",ky,"2019-J., 13-DECEMBER",None
8,1184,"<html lang=""kk"" dir=""ltr"" class="" js""><head>\n...",2021 Ж. 20 APRIL,kk,"<img alt="" "" src=""/sites/g/files/zskgke326/fil...",2021 Ж. 20 APRIL,kk,"2021 , 20.08.2010",None
9,1283,"<html lang=""ky"" dir=""ltr"" class="" js""><head>\n...","2020-Ж., 10-APRIL",ky,Постер: БУУӨП Кыргызстан <b><i>“Ачык Инновац...,"2020-Ж., 10-APRIL",ky,"2020-J., 10-APRIL",None


In [11]:
# translator = Translator(from_lang = 'es', to_lang = 'en')
# translator.translate(r'21 DE FEBREERO DE 2022')
# ts.translate_text('2 DE MAYO DE 2023', from_language = 'es', to_language = 'en')

In [12]:
cur = conn.cursor()

for i, r in body[body['datetime'].notnull()].iterrows():
    sql = """
        UPDATE articles
        SET parsed_date = %s
        WHERE id = %s
    ;"""
    cur.execute(sql, (r['datetime'], r['id']))

conn.commit()
cur.close()

In [13]:
## CLOSE THE CONNECTION TO THE DB
conn.close()